These are my notes and little tests while learning Google's machine learning library, TensorFlow.

TensorFlow can now only be installed in Mac OS and Linux. I'm using virtual machine running Ubuntu on Windows 10.

First one are some basic usage of TensorFlow.

The second is a "multinomial logistic regression".

The third part is more thorough with a little test.

Finally, the last one is a "convolutional neural network".

Below are some basic usage of TensorFlow.

In [2]:
import tensorflow as tf

def Session():
    matrix_1 = tf.constant([[3.0, 3.0]]) #1x2 matrix
    matrix_2 = tf.constant([[2.0], [2.0]]) #2x1 matrix
    product = tf.matmul(matrix_1, matrix_2)

    with tf.Session() as sess:
        print(sess.run(product))

def Varialbes():
    state = tf.Variable(0)
    one = tf.constant(1)
    new_state = tf.add(state, one)
    update = tf.assign(state, new_state)
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        sess.run(init)
        print(sess.run(state))
        for i in range(3):
            sess.run(new_state)
            sess.run(update)
            print(sess.run(state))

def Fetches():
    one = tf.constant([1.0])
    two = tf.constant([2.0])
    three = tf.constant([3.0])
    sum = tf.add(one, two)
    product = tf.mul(sum, three)

    with tf.Session() as sess:
        print(sess.run([sum, product]))

def Feeds():
    input_1 = tf.placeholder(tf.float32)
    input_2 = tf.placeholder(tf.float32)
    output = tf.mul(input_1, input_2)

    with tf.Session() as sess:
        print(sess.run([output], feed_dict={input_1:[3.0], input_2:[5.0]}))

Session()
Varialbes()
Fetches()
Feeds()

[[ 12.]]
0
1
2
3
[array([ 3.], dtype=float32), array([ 9.], dtype=float32)]
[array([ 15.], dtype=float32)]


Below is a quick implementaion of multinomial logistic regression used in MNIST.

MNIST is a simple computer vision dataset. It consists of images of handwritten digits.

We're going to train the model to look at images and predict what digits they are.

The accuracy is around 90%. It's very bad.

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10])) # x * W = None * 10
b = tf.Variable(tf.zeros([10])) # 1 * 10
y = tf.nn.softmax(tf.matmul(x, W) + b) # probability distribution (0 < y < 1)

y_ = tf.placeholder(tf.float32, [None, 10]) # true distribution (one-hot vector)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=1))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_x, batch_y = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_x, y_: batch_y})
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)) # argmax: return the largest value of a tensor, dimension to reduce
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # cast: change data type to float32
    print('The accuracy is %.2f' % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
The accuracy is 0.92


Here is a thorough implementation of multinomial logistic regression alse used in MNIST.

However, this time I draw three 7 * 7 matrices which is reshaped to 28 * 28 and let the model to guess the numbers.

The result isn't quite accurate, but I'm satisfied with it.

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # one-hot vector, label
W = tf.Variable(tf.zeros([784,10])) # 784 inputs and 10 outputs
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
sess.run(tf.initialize_all_variables())

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]}) # replace placeholder wdwith training samples, x: handwriting, y_: answer



import numpy as np

test0 = np.array([0,0,0,0,0,0,0,
                  0,0,1,1,1,0,0,
                  0,1,0,0,0,1,0,
                  0,1,0,0,0,1,0,
                  0,1,0,0,0,1,0,
                  0,0,1,1,1,0,0,
                  0,0,0,0,0,0,0])

test1 = np.array([0,0,0,1,0,0,0,
                  0,0,0,1,0,0,0,
                  0,0,0,1,0,0,0,
                  0,0,0,1,0,0,0,
                  0,0,0,1,0,0,0,
                  0,0,0,1,0,0,0,
                  0,0,0,1,0,0,0])

test3 = np.array([0,0,1,1,1,0,0,
                  0,1,0,0,0,1,0,
                  0,0,0,0,0,1,0,
                  0,0,1,1,1,0,0,
                  0,0,0,0,0,1,0,
                  0,1,0,0,0,1,0,
                  0,0,1,1,1,0,0])

def drawer(test):
    sketch_1 = []
    sketch_2 = []
    picture_1 = []
    picture_2 = []
    for i in test:
        for j in range(4):
            sketch_1.append(i)
    for k in range(7):
        sketch_2.append(sketch_1[k*28:k*28+28])
    for l in range(len(sketch_2)):
        for m in range(4):
            picture_1.append(sketch_2[l])
    print('input:')
    print(np.array(picture_1))
    for n in picture_1:
        for o in n:
            picture_2.append(o)
    return picture_2

label = [[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0,0],
         [0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,1,0,0],
         [0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,1]]

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)) # argmax(): return the index of the highest entry along some axis
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test = drawer(test3)
for n in label:
    if accuracy.eval(feed_dict={x: [test], y_: [n]}) == 1:
        print('This is %d.' % label.index(n))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
input:
[[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 

This one is a simple convolutional neural network.

I finally get to the deep learning.

Those comments are my notes. Ignore it or read it all!

It's trained by MNIST, too.

However, the highest accuracy can get to 100%!

With a lot, a lot of time......

In [10]:
import tensorflow as tf

##############################################################
'''Train and Evaluate the Model'''                           #
from tensorflow.examples.tutorials.mnist import input_data   #
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)#
                                                             #
sess = tf.InteractiveSession()                               #
                                                             #
x = tf.placeholder(tf.float32, shape=[None, 784])            #
y_ = tf.placeholder(tf.float32, shape=[None, 10])            #
sess.run(tf.initialize_all_variables())                      #
##############################################################

# Initialize weights with a little noise for symmetry breaking
# and preventing 0 gradients.
def weight_variable(shape):
    # Outputs random values from a truncated normal distribution.
    initial = tf.truncated_normal(shape=shape, stddev=0.1) # standard deviation
    return tf.Variable(initial)

# Initialize biases with slightly positive numbers to avoid "dead neurons".
def bias_variable(shape):
    initial = tf.constant(value=0.1, shape=shape)
    return tf.Variable(initial)

# Computes a 2D convolution from 4D input and filter tensors.
def conv2d(x, W):
    # input: [batch, in_height, in_width, in_channels]
    # filter: [filter_height, filter_width, in_channels, out_channels]
    # 1. Flattens the filter to a 2D matrix with shape [filter_height * filter_width * in_channels, output_channels].
    # 2. Extracts image patches from the input tensor to form a tensor of shape [batch, out_height, out_width, filter_height * filter_width * in_channels].
    # 3. output = 2D filter * image patch
    # strides: a list of ints, 1D, length 4
    # ????????????????????????????????????????????????????????????????????????????????????????????????????
    return tf.nn.conv2d(input=x, filter=W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # value: a 4D tensor with shape [batch, height, width, channels]
    # ksize: a list of ints that has length >= 4
    # strides:  a list of ints that has length >= 4
    return tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

'''First Convolutional Layer: convolution >> max pooling'''
# 32 features for each 5x5 patch
W_conv_1 = weight_variable([5, 5, 1, 32]) # [height, width, input channels, output channels]
b_conv_1 = bias_variable([32])
# Reshape x to 28 * 28.
# 1: number of color channels
# -1: Total size remains constant.
x_image = tf.reshape(x, [-1,28,28,1])
# relu: rectified linear unit
# rectified linear: an activation function defined as f(x) = max(0, x)
# Its smooth approximation is the analytic function defined as f(x) = ln(1 + e^x) (softplus)
h_conv_1 = tf.nn.relu(conv2d(x_image, W_conv_1) + b_conv_1)
h_pool1 = max_pool_2x2(h_conv_1)

'''Second Convolutional Layer: convolution >> max pooling'''
# 64 features for each 5x5 patch
W_conv_2 = weight_variable([5, 5, 32, 64])
b_conv_2 = bias_variable([64])
h_conv_2 = tf.nn.relu(conv2d(h_pool1, W_conv_2) + b_conv_2)
h_pool_2 = max_pool_2x2(h_conv_2)

'''Densily Connected Layer: Add 1024 fully-connected neurons to process the entire image.'''
# Image size is 7 * 7 now.
W_fc_1 = weight_variable([7 * 7 * 64, 1024])
b_fc_1 = bias_variable([1024])
# Reshape the tensor from the pooling layer into a batch of vectors.
h_pool_2_flat = tf.reshape(h_pool_2, [-1, 7*7*64])
h_fc_1 = tf.nn.relu(tf.matmul(h_pool_2_flat, W_fc_1) + b_fc_1)

'''Drop-out Layer: To reduce overfitting.'''
keep_prob = tf.placeholder(tf.float32)
h_fc_1_drop = tf.nn.dropout(h_fc_1, keep_prob)

'''Readout Layer: Softmax'''
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc_1_drop, W_fc2) + b_fc2)

'''Train and Evaluate the Model'''
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f9e28b37ba8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 171, in __del__
    self.close()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 976, in close
    self._default_session.__exit__(None, None, None)
  File "/usr/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/ops.py", line 3378, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


step 0, training accuracy 0.14
step 100, training accuracy 0.84
step 200, training accuracy 0.94
step 300, training accuracy 0.88
step 400, training accuracy 0.98
step 500, training accuracy 0.9
step 600, training accuracy 1
step 700, training accuracy 0.96
step 800, training accuracy 0.86
step 900, training accuracy 1
step 1000, training accuracy 0.94
step 1100, training accuracy 1
step 1200, training accuracy 0.98
step 1300, training accuracy 1
step 1400, training accuracy 0.98
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 0.96
step 1800, training accuracy 0.96
step 1900, training accuracy 1
step 2000, training accuracy 0.94
step 2100, training accuracy 1
step 2200, training accuracy 1


KeyboardInterrupt: 